# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [11]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [12]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [13]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [14]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [15]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [21]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [17]:
gamma = 1
sigma = .005

In [26]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [19]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [20]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.2%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 2:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 3:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,044.00 (365 días después).


Pregunta 4:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .1.4756 .


Pregunta 5:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 6:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
for

## Respuestas

### Pregunta 1

In [37]:
strikes = .982
print('El valor de la Call con strike 0.982 es',round(hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ),8))

El valor de la Call con strike 0.982 es 0.0065627


### Pregunta 2

La formula para calcular Call y Put sobre bono cero cupon son las mismas para HW y Vasicek, ya que el modelo de HW es una extensión de Vasicek, pero HW es capaz de ajustar la tasa del bono cero cupon, no así Vasicek

### Pregunta 3

In [41]:
ICP0=10000.00
ICPT=10044.00
D=365
TNA=((ICPT/ICP0)-1)*(360/365)
print('El valor de TNA es:',round(TNA,4))
#El valor de TNA es de 0.0043

El valor de TNA es: 0.0043


### Pregunta 4

In [60]:
from typing import List, Tuple
import plotly.express as px
gamma = 1
sigma = .005
r_ = .005
r0 =0.000811 # primera tasa que aparece

def vasicek_path(r0: float,
                 gamma: float, r_: float,
                 sigma: float,
                 num_dias: int = 263,
                 dias_agno: int = 264,
                 pasos_dia: int = 1) -> List[Tuple[float, float]]:

    dt = 1 / 264
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * 0.14756 # Discretización de Euler
        result.append((i * dt, r))
    return result
#N(0,1) = .14756

sim = vasicek_path(r0, gamma, r_, sigma, num_dias=1)
print(sim)



[(0, 0.000811), (0.003787878787878788, 0.0008722758752647001)]


### Pregunta 5

La diferencia entre valorizar por MonteCarlo por HW a tasa libre de riesgo y medida Forward es que para poder calcular una medida Forward hay que calcular un drift en theta, donde se resta el valor B(t,T)*sigma, además existe una diferencia en la forma de traer a valor presente, ya que al momento de traer a valor presente, en medida forward se multiplica el payoff por el factor de descuento a T, en el caso de tasa libre de riesgo se utiliza descontando en base a la curva, y la forma de ver que el resultado es correcto, es ver que la simulación se parece a df de la curva

### Pregunta 6

In [40]:
#Tiempos
t1=382
t2=554

r1=curva(t1)
r2=curva(t2)
df1=math.exp(-r1*t1/365)
df2=math.exp(-r2*t2/365)
fwd12=(df1/df2-1.0)*360.0/(t2-t1)
print('la tasa forward es:',fwd12)
print('La tasa forward en porcentaje es:',f'{fwd12:.16%}')

la tasa forward es: 0.0005032025299008271
La tasa forward en porcentaje es: 0.0503202529900827%


### Pregunta 7